# Using the NER model to get unique ingredients from a post

In [1]:
import sys
sys.path.append("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta")
import functions

import numpy as np
import pandas as pd
import re

In [2]:
posts1 = pd.read_csv("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta/Instagram Data Scraping/Phantom Buster/recipe_posts.csv")
posts2 = pd.read_csv("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta/Instagram Data Scraping/Phantom Buster/recipe_posts_2.csv")
posts3 = pd.read_csv("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta/Instagram Data Scraping/Phantom Buster/recipe_posts_3.csv")

posts = pd.concat([posts1,posts2,posts3])

In [3]:
# Removing duplicate posts
posts = functions.remove_duplicates(posts)

In [4]:
# Getting english posts
posts = functions.get_english_posts(posts)

Detecting language of each post...
Language detection complete.
Time taken: 0:07:39.214400


In [5]:
# Preprocessing text descriptions
posts['description_preprocessed'] = posts['description'].apply(functions.preprocess_text)

In [7]:
# Using NER model to add unique ingredients list to df
posts = functions.include_unique_ingredients(posts, "./model-best")

NameError: name 'nlp' is not defined

In [8]:
def get_unique_ingredients(text, model):
    
    nlp = spacy.load(model)
    doc = nlp(text)
    
    ingredients = []
    ingredients_singularized = []
    
    for entity in doc.ents:
        if entity.label_ == 'INGREDIENT':
            ingredients.append(entity.text)

    ingredients = np.unique(ingredients)
    
    # Singularizing ingredients using inflect python package
    p = inflect.engine()
    for word in ingredients:
        if p.singular_noun(word):
            ingredients_singularized.append(p.singular_noun(word))
        else:
            ingredients_singularized.append(word)
            
    return np.unique(ingredients_singularized)

In [12]:
pd.read_csv('preprocessed_descriptions.csv').loc[0,'description_preprocessed']

'chocolate fudge protein oatmeal \ningredients:\n1/2 cup instant oats\n1/2 cup milk\n1/2 cup water\n1 scoop chocolate protein powder\n1 teaspoon brown sugar\n\ntoppings:\n1 tablespoon peanut butter\n1 tablespoon chocolate spread\n1/2 a banana\nprotein bar .in'

In [16]:
import spacy, inflect

In [17]:
get_unique_ingredients(pd.read_csv('preprocessed_descriptions.csv').loc[0,'description_preprocessed'], "./model-best")

array(['banana', 'milk', 'oat', 'peanut butter', 'sugar', 'water'],
      dtype='<U13')

In [19]:
test_df = pd.read_csv('preprocessed_descriptions.csv')
test_df['unique_ingredients'] = test_df['description_preprocessed'].apply(get_unique_ingredients, args=("./model-best",))

In [30]:
print(test_df.loc[200, 'description_preprocessed'])
print(test_df.loc[200, 'unique_ingredients'])

sausage, mushroom & brie egg bake

ingredients:
- 16 ounce mild italian sausage
- 2 cup mushroom (sliced)
- 1 small red onion (diced)
- 1 clove minced garlic
- 4 cup spinach
- 1 cup brie cheese (cubed)
- 10 eggs
- 1/2 cup milk
- 1 1 /2 teaspoon garlic powder
- 1 1 /2 teaspoon onion powder
- 1 teaspoon green goddess seasoning (can omit)
- salt
- pepper
- 1 can crescent roll

steps:
1 . start by preheating the oven to 350 degrees. while the oven is preheating, brown the sausage, slice the mushrooms, and dice the onion.
2 . once the sausage is done, set aside and add the mushroom, onion, and garlic to the pan with the grease from the sausage. stir frequently until it is all soft and fragrant, then add in the spinach until it is wilted. set aside.
3 . next dice up the brie cheese into cubes and also set aside. grab a medium sized bowl and add the eggs, milk, garlic powder, onion powder, salt, pepper, and green goddess seasoning. mix until well combined.
4 . once everything is has been prep

In [33]:
for index, post in enumerate(test_df['description_preprocessed']):
    if "what could be better in summer " in post:
        print(index)

121
285


In [38]:
list(test_df['unique_ingredients'][121])

['cheese',
 'chili',
 'cream',
 'funnel',
 'garlic powder',
 'oregano',
 'paprika',
 'pepper',
 'salt',
 'yogurt']

In [40]:
print(test_df['description_preprocessed'][121])

what could be better in summer than a bbq with your favorite people? we've brought you a particularly cool recipe idea for this: pulled pork carnitas with our homemade sour cream sauce! 

ingredients:
- 250 gram sour cream
- 100 gram mayonnaise (min. 50 % fat)
- 100 gram cream cheese
- 50 gram yogurt 3.2 %
- 1/2 teaspoon smoked paprika powder
- 1/2 teaspoon chili cayenne
- 1 teaspoon ground cumin
- 1/2 teaspoon garlic powder
- 1 tablespoon dried oregano
- juice of 1/4 lime
- salt & pepper

preparation:
 blend all ingredients finely with a hand blender or in a stand mixer.
then pour the entire mixture through isi funnel & sieve into a 0.5 litre isi whip. screw on an isi professional charger and shake vigorously 12-14 times.
